In [1]:
from utils import BCC_TOOLS_PATH
from depsurf import BPFTOOL_SRC_PATH

In [5]:
!make -C {BCC_TOOLS_PATH}

make: Entering directory '/Users/szhong/Code/DepSurf/csrc/bcc/libbpf-tools'
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Code/DepSurf/csrc/bcc/libbpf-tools'


In [2]:
!patch --forward --directory={BPFTOOL_SRC_PATH} < csrc/bpftool.patch

patching file gen.c


In [5]:
!make -C {BPFTOOL_SRC_PATH} bpftool

make: Entering directory '/Users/szhong/Code/DepSurf/csrc/bpftool/src'
...                        libbfd: [ OFF ]
...               clang-bpf-co-re: [ on  ]
...                          llvm: [ on  ]
...                        libcap: [ OFF ]
make: 'bpftool' is up to date.
make: Leaving directory '/Users/szhong/Code/DepSurf/csrc/bpftool/src'


In [4]:
!git -C {BPFTOOL_SRC_PATH} reset --hard

HEAD is now at c428db1 sync: Pull latest bpftool changes from kernel


In [3]:
%reload_ext autoreload
%autoreload 2

In [6]:
from depsurf import BPFProgram, VersionGroup, gen_report, PROJ_PATH
from utils import iter_bcc_objects, REPORT_PATH, save_pkl


REPORT_PATH.mkdir(parents=True, exist_ok=True)
GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH]

paths = list(iter_bcc_objects())
paths.append(PROJ_PATH / "csrc" / "tracee" / "dist" / "tracee.bpf.o")

results = {}
for path in paths:
    bpf = BPFProgram(path)
    result_path = REPORT_PATH / f"{bpf.name}.log"
    results[bpf.name] = gen_report(bpf.deps, GROUPS, path=result_path)


save_pkl(results, "bcc")

[       system.py:19 ] INFO: Running command: "/Users/szhong/Code/DepSurf/csrc/bpftool/src/bpftool  gen min_core_btf /Users/szhong/Code/DepSurf/csrc/bcc/libbpf-tools/.output/hardirqs.bpf.o /Users/szhong/Code/DepSurf/csrc/bcc/libbpf-tools/.output/hardirqs.bpf.min.btf /Users/szhong/Code/DepSurf/csrc/bcc/libbpf-tools/.output/hardirqs.bpf.o"


[       system.py:19 ] INFO: Running command: "/Users/szhong/Code/DepSurf/csrc/bpftool/src/bpftool btf dump file /Users/szhong/Code/DepSurf/csrc/bcc/libbpf-tools/.output/hardirqs.bpf.min.btf --json > /Users/szhong/Code/DepSurf/csrc/bcc/libbpf-tools/.output/hardirqs.bpf.min.btf.json"
[       system.py:19 ] INFO: Running command: "/Users/szhong/Code/DepSurf/csrc/bpftool/src/bpftool btf dump file /Users/szhong/Code/DepSurf/csrc/bcc/libbpf-tools/.output/hardirqs.bpf.min.btf format raw > /Users/szhong/Code/DepSurf/csrc/bcc/libbpf-tools/.output/hardirqs.bpf.min.btf.txt"
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Code/DepSurf/data/btf_norm/4.4.0-21-generic-amd64.pkl
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Code/DepSurf/data/btf_norm/4.8.0-22-generic-amd64.pkl
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Code/DepSurf/data/btf_norm/4.10.0-19-generic-amd64.pkl
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Code/DepSurf/data/btf_norm

In [10]:
from utils import BCC_CATEGORY, Category, iter_bcc_objects

categories = {c: [] for c in Category}

for path in iter_bcc_objects():
    name = path.name.removesuffix(".bpf.o")
    c = BCC_CATEGORY[name]
    categories[c].append(name)

for c in categories:
    print(f"{c:12} ({len(categories[c]):2}): {', '.join(categories[c])}")

CPU          (11): hardirqs, wakeuptime, offcputime, runqlat, runqlen, profile, softirqs, cpufreq, cpudist, llcstat, runqslower
Process      ( 4): futexctn, exitsnoop, sigsnoop, execsnoop
Memory       ( 5): oomkill, numamove, slabratetop, drsnoop, memleak
Block Device ( 7): biotop, bitesize, biolatency, biostacks, biopattern, mdflush, biosnoop
Filesystem   (11): filelife, opensnoop, syncsnoop, cachestat, fsdist, vfsstat, mountsnoop, statsnoop, filetop, fsslower, readahead
Network      (11): solisten, tcptop, tcpconnlat, tcprtt, tcplife, tcpconnect, tcpstates, tcpsynbl, bindsnoop, tcptracer, tcppktlat
Userspace    ( 3): javagc, gethostlatency, bashreadline
Other        ( 5): funclatency, ksnoop, klockstat, syscount, capable
